## Dacon  14회 KB 금융문자 분석 모델링 경진대회
## dacon (팀명)
## 2020년 12월 24일 (제출날짜)

# 모델링 코드 작성방법

A 코드 관련

1) 입상자는 코드 제출 필수. 제출 코드는 예측 결과를 리더보드 점수로 복원할 수 있어야 함

2) 코드 제출시 확장자가 R user는 R or .rmd. Python user는 .py or .ipynb

3) 코드에 ‘/data’ 데이터 입/출력 경로 포함 제출 or R의 경우 setwd(" "), python의 경우 os.chdir을 활용하여 경로 통일

4) 전체 프로세스를 일목요연하게 정리하여 주석을 포함하여 하나의 파일로 제출

5) 모든 코드는 오류 없이 실행되어야 함(라이브러리 로딩 코드 포함되어야 함).

6) 코드와 주석의 인코딩은 모두 UTF-8을 사용하여야 함

 
B 외부 데이터 관련

1) 외부 공공 데이터 (날씨 정보 등) 사용이 가능하나, 코드 제출 시 함께 제출

2) 공공 데이터 외의 외부 데이터는 법적인 제약이 없는 경우에만 사용 가능

3) 외부 데이터를 크롤링할 경우, 크롤링 코드도 함께 제출

## 1. 라이브러리 및 데이터
## Library & Data

In [1]:
import numpy as np
import pandas as pd
import joblib
import re
import random
import warnings
warnings.filterwarnings(action='ignore')
import os
os.chdir('../0_Data')

from collections import namedtuple, Counter
from tqdm import tqdm
from gensim.models import doc2vec
from konlpy.tag import Okt

# 데이터 로드
train=pd.read_csv("train.csv")
test_data=pd.read_csv("public_test.csv")

## Crawling Code(크롤링 진행 시 기입)

In [2]:
# 없음

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing

In [3]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(train[train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 11750 )

random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

train_data=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

In [4]:
okt = Okt()

removeword=['[Web발신]', "(광고)", "\n", "\r",'X','XX','XXX','XXXX','XXXXX','XXXXXX','XXXXXXX',
            '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ','..', '.(', '은', '는'] #그냥 사용시 토큰으로 같이 인식됨
# X를 추가한 이유 : 제거 안하니까 kbXXX이런식으로 같이 인식해버림, 굳이 그렇게 인식 될 이유가 전혀 없어보였음

def ishangul(text):
    """해당 글자가 한글인지 찾아주는 함수입니다."""
    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', text))
    return hanCount > 0

def blank(text):
    """한글과 한글이 아닌 단어사이에 공백을 삽입합니다."""
    return_doc = []
    pre_hangul = True
    for i in text:
        hangul = ishangul(i)
        if pre_hangul != hangul:
            return_doc.append(" ")
            return_doc.append(i)
        else:
            return_doc.append(i)
        pre_hangul = hangul
    return "".join(return_doc)

def tokenize(doc, norm=True, stem=True, removeword=None):
    """
    공백 삽입을 찾아내고, 이후 doc2vec 을 하기위한 데이터로 바꾸줍니다.
    특수문자를 'Punctuation' 으로 바꿉니다.
    외국어와 숫자를 제거합니다.
    """
    if removeword:
        for word in removeword:
            doc = doc.replace(word, " ") #없앨 단어 대체
    toekns = ["/".join(word) for word in okt.pos(doc)] #토큰 화
    doc = blank(doc)  # 해당 문자에서 영어와 한글 사이에 공백을 넣는 함수
    
    adj_toekns = []
    for t in toekns:
        if "Punctuation" in t:
            adj_toekns.append("Punctuation")
        elif "Foreign" in t: # 외국어 제거
            pass
        elif "Number" in t:  # 숫자 제거
            pass
        else:
            adj_toekns.append(t) 
    return adj_toekns #외국어, 숫자 제거하고 특수문자는 Punctuation 자체로 바꾸고, url도 해당 url 특징에 맞게 바꾸는 함수


#위에 주어진 함수를돌리기 위해서 array화 실행
train_data_list = np.array(train_data[["text", "smishing"]].values.tolist())

train_docs = [ (tokenize( row[0], removeword=removeword ), row[1] ) for row in tqdm(train_data_list)]

test_data['smishing']=2

test_data_list = np.array(test_data[["text","smishing"]].values.tolist())

test_docs = [ ( tokenize(row[0],  removeword=removeword ), row[1] ) for row in tqdm(test_data_list) ]

100%|██████████████████████████████████████████████████████████████████████████████| 1626/1626 [00:21<00:00, 74.71it/s]


In [5]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs] # d는 단어랑 타입, c는 스미싱 여부 라벨링
tagged_test_docs = [TaggedDocument(d, [c]) for d, c in test_docs] # d는 단어랑 타입, c는 스미싱 여부 라벨링(test에서는 없으니 임의로 설정된 것)

## 3. 탐색적 자료분석
## Exploratory Data Analysis


In [6]:
# 생략

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling

In [7]:
# 생략

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [8]:
os.chdir('../1_Model')

doc_init = doc2vec.Doc2Vec(size=1000, window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025, iter=2, seed = 2019) 
doc_init.build_vocab(tagged_train_docs) 

doc_init.train(tagged_train_docs, epochs=doc_init.iter, total_examples=doc_init.corpus_count)

doc_init.save('seven.doc2vec')

In [9]:
def t1(tt):
    doc_init.random.seed(0) #동일한 시드를 놔둬서 같은 array의 값이 나오게 설정해둠
    return doc_init.infer_vector(tt.words) #단어를 vector화 시킴

In [10]:
train_x=[ t1(i) for i in tagged_train_docs ]

test_x=[ t1(i) for i in tagged_test_docs ]

In [11]:
#스미싱 링크 라벨

train_select = train_data["smishing"]# 스미싱 링크 여부랑 스미싱 여부만 가져오기

train_embedding = pd.DataFrame(train_x) #vector to dataframe
train_tf = pd.concat([train_embedding, train_select], axis=1)

test_tf = pd.DataFrame(test_x)

In [12]:
train_data2=train_tf.copy()
test_data2=test_tf.copy()

In [13]:
from lightgbm import LGBMClassifier

In [14]:
lgbm = LGBMClassifier(n_estimators=500, learning_rate=0.025,

                     max_depth=7,

                     min_child_samples=100,

                     random_state=1234)

In [15]:
train_xx=train_data2.copy()
train_y = train_xx['smishing']
del train_xx['smishing']

In [16]:
lgbm.fit(train_xx, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.025, max_depth=7,
               min_child_samples=100, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=31,
               objective=None, random_state=1234, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [17]:
y_pred_ten=lgbm.predict_proba(test_data2)

In [18]:
proba = [ i[1] for i in y_pred_ten ]

In [19]:
test_data['pred']=proba

In [20]:
joblib.dump(lgbm, 'lgb.pkl')

['lgb.pkl']

## 6. 결과 및 결언
## Conclusion & Discussion

In [21]:
# 생략